In [1]:
import pandas as pd 
departments=pd.read_csv(r"C:\Users\Hp laptop\OneDrive\Documents\project\New folder\data\raw\departments.csv")
orders=pd.read_csv(r"C:\Users\Hp laptop\OneDrive\Documents\project\New folder\data\raw\orders.csv")
order_products=pd.read_csv(r"C:\Users\Hp laptop\OneDrive\Documents\project\New folder\data\raw\Order_products__prior.csv")
products=pd.read_csv(r"C:\Users\Hp laptop\OneDrive\Documents\project\New folder\data\raw\Products.csv")
aisles=pd.read_csv(r"C:\Users\Hp laptop\OneDrive\Documents\project\New folder\data\raw\Aisles.csv")

In [2]:
print("DATA LOADED SUCCESSFULLY")

DATA LOADED SUCCESSFULLY


In [3]:
print("We performed memory-efficient left joins starting from the order-product interaction" 
" table to preserve all purchase events, incrementally enriching them with order-level0, product-level, aisle,"
" and department metadata")

We performed memory-efficient left joins starting from the order-product interaction table to preserve all purchase events, incrementally enriching them with order-level0, product-level, aisle, and department metadata


In [4]:
orders= pd.read_csv(r"C:\Users\Hp laptop\OneDrive\Documents\project\New folder\data\raw/orders.csv",dtype = {'order_id': 'int32',
                                                                        'user_id': 'int32',
                                                                       'order_number': 'int16'})
##I made join for the  data 
df=order_products.merge(orders,on='order_id',how='left')
df=df.merge(products,on='product_id',how='left')
df=df.merge(aisles,on='aisle_id',how='left')
df=df.merge(departments,on='department_id',how='left')
## JOINING COMPLETED SUCCESSFULLY
print("the final dataFrame shape is :", df.shape)

KeyError: 'order_id'

In [ ]:
order_products.columns

Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')

In [ ]:
orders.columns

Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')

In [ ]:
##check the data:
print(df.head())
print(df.info())
print(df.columns)

NameError: name 'df' is not defined

In [ ]:
## cleaning the data:
missing_values=df.isnull().sum()
print("missing values in each column:\n",missing_values)
df["days_since_prior_order"]=df["days_since_prior_order"].fillna(0).astype("int16")

missing values in each column:
 order_id                  0
product_id                0
add_to_cart_order         0
reordered                 0
user_id                   0
eval_set                  0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
product_name              0
aisle_id                  0
department_id             0
aisle                     0
department                0
dtype: int64


In [ ]:
##outlier detection and treatment :
num_cols=[
    'add_to_cart_order',
    'order_number',
    'order_hour_of_day',
    'days_since_prior_order'
]
df[num_cols].describe()

,add_to_cart_order,order_number,order_hour_of_day,days_since_prior_order
count,3.243449e+07,3.243449e+07,3.243449e+07,32434489.0
mean,8.351076e+00,1.714205e+01,1.342498e+01,0.0
std,7.126671e+00,1.753504e+01,4.246365e+00,0.0
min,1.000000e+00,1.000000e+00,0.000000e+00,0.0
25%,3.000000e+00,5.000000e+00,1.000000e+01,0.0
50%,6.000000e+00,1.100000e+01,1.300000e+01,0.0
75%,1.100000e+01,2.400000e+01,1.600000e+01,0.0
max,1.450000e+02,9.900000e+01,2.300000e+01,0.0


In [ ]:
print("final dataFrame info after cleaning:")

In [ ]:
#1️⃣ User-level features 
#groupby على user_id و count للـ order_id
user_order_count=orders.groupby('user_id')['order_id'].count().reset_index()
# نغير اسم العمود عشان يكون واضح
user_order_count.rename(columns={'order_id':'user_order_count'},inplace=True)
users=users.merge(user_order_count,on='user_id',how='left')

users_total_spent=orders.groupby('user_id')['total_amount'].sum().reset_index()
users_total_spent.rename(columns={'total_amount':'total_spent'},inplace=True)
users=users.merge(users_total_spent,on='user_id',how='left')

users_avg_spent=orders.groupby('user_id')['total_amount'].mean().reset_index()
users_avg_spent.rename(columns={'total_amount':'avg_spent'},inplace=True)
users=users.merge(users_avg_spent,on='user_id',how='left')


# نحول العمود لتاريخ لو مش converted
orders['order_data']=pd.to_datetime(orders['order_data'])

# نحدد آخر تاريخ لكل مستخدم
users_last_order=orders.groupby('user_id')['order_data'].max().reset_index()
users_last_order.rename(columns={'order_data':'last_order_date'},inplace=True)
users=users.merge(users_last_order,on='user_id',how='left')

# نفترض أن اليوم المرجعي هو آخر تاريخ موجود بالداتا
todays=orders[;order_data].max()
# نحسب الأيام منذ آخر طلب
user_last_order['user_last_order_days_ago'] = (today - user_last_order['last_order_date']).dt.days

# دمج مع جدول المستخدمين
users = users.merge(user_last_order[['user_id','user_last_order_days_ago']], on='user_id', how='left')

In [ ]:
#2️⃣ Product-level features
products_ordered_count=order_products.groupby('product_id')['order_id'].count().reset_index()
products_ordered_count.rename(columns={'order_id':'products_ordered_count'},inplace=True)
products=products.merge(products_ordered_count,on='product_id',how='left')

product_avg_price=products.groupby('product_id')['price'].mean().reset_index()
product_avg_price.rename(columns={'price':'product_avg_price'},inplace=True)
products=products.merge(product_avg_price,on='product_id',how='left')

product_unique_users=order_products.goupby('product_id')['user_id'].nunique().reset_index()
product_unique_users.rename(columns={'user_id':'unique_user_count'},inplace=True)
products=products.merge(product_unique_users,on='product_id',how='left')

In [ ]:
#3️⃣ User×Product-level feature
user_product_order_count=order_products.groupby(['user_id','product_id'])['order_id'].count().reset_index()
user_product_order_count.rename(columns={'order_id':'user_product_order_count'},inplace=True)
order_products=order_products.merge(user_product_order_count,on=['user_id','product_id'],how='left')

order_products['order_data']=pd.to_datetime(order_products['order_data'])
user_product_last_order=order_products.groupby(['user_id','product_id'])['order_data'].max().reset_index()
user_product_last_order.rename(columns={'order_data':'user_product_last_order_date'},inplace=True)

# اليوم المرجعي
today = order_products['order_date'].max()

# الأيام منذ آخر طلب
user_product_last_order['user_product_last_order_days_ago'] = (today - user_product_last_order['last_order_date']).dt.days

order_products=order_products.merge(user_product_last_order,on=['user_id','product_id'],how='left')

In [ ]:
user_product_avg_spent = order_products.groupby(['user_id','product_id'])['price'].mean().reset_index()
user_product_avg_spent.rename(columns={'price':'user_product_avg_spent'}, inplace=True)

# دمج مع جدول order_products
order_products = order_products.merge(user_product_avg_spent, on=['user_id','product_id'], how='left')

In [ ]:
orders['order_data']=pd.to_datetime(orders['order_data'])
# استخراج يوم الأسبوع
orders['order_day_of_week'] = orders['order_data'].dt.dayofweek
orders['order_month'] = orders['order_data'].dt.month
# ترتيب الطلبات لكل مستخدم حسب التاريخ
orders=orders.sort_values(['user_id','order_data'])
# حساب الفرق بين الطلبات السابقة
orders['day_since_last_order']=orders.groupby('user_id')['order_data'].diff().dt.days

# إذا أول طلب، ممكن يكون NaN → نملأه بصفر أو قيمة كبيرة
orders['day_since_last_order']=orders['day_since_last_order'].fillna(0).astype('int16')

In [ ]:
users_last_order=orders.groupby('user_id')['order_data'].max().reset_index()
users_last_order.rename(columns={'order_data':'last_order_date'},inplace=True)
# اليوم المرجعي (آخر تاريخ موجود في الداتا)
today = orders['order_data'].max()

# أيام منذ آخر طلب
users_last_order['user_recency_days'] = (today - users_last_order['last_order_date']).dt.days
# دمج مع جدول المستخدمين
users = users.merge(users_last_order[['user_id','user_recency_days']], on='user_id', how='left')
# عدد الطلبات لكل مستخدم
users_Frequency=orders.groupby('user_id')['order_id'].count().reset_index()
users_Frequency.rename(columns={'order_id':'user_Frequency'},inplace=True)
# عدد الطلبات لكل مستخدم
users=users.merge(users_Frequency,on='user_id',how='left')

#3️⃣ Frequency خلال فترة زمنية محددة (اختياري)

# إذا بدك تحسبي عدد الطلبات في آخر 30 يوم لكل مستخدم:
# اليوم المرجعي
today = orders['order_data'].max()

# فلترة الطلبات خلال آخر 30 يوم
last_30_days = orders[orders['order_data'] >= today - pd.Timedelta(days=30)]

# حساب عدد الطلبات لكل مستخدم خلال آخر 30 يوم
user_freq_30d = last_30_days.groupby('user_id')['order_id'].count().reset_index()
user_freq_30d.rename(columns={'order_id':'user_frequency_30d'}, inplace=True)

# دمج مع جدول المستخدمين
users = users.merge(user_freq_30d, on='user_id', how='left')


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
products['product_category']=le.fit_transform(products['product_category'])
#2️⃣ One-Hot Encoding
products=pd.get_dummies(products,columns=['product_category'],prefix='category')

#3️⃣ Target Encoding
#فترض أن الهدف هو 'order_amount'
target_mean=ordersroupby('prodect_category')['order_amount'].mean().reset_index()
target_mean.rename(columns={'order_amount':'category_target_mean'},inplace=True)
# دمج مع جدول المنتجات
products = products.merge(target_mean, on='product_category', how='left')